In [54]:
import sys
import matplotlib.pyplot as plt
import sklearn
assert sys.version_info >= (3, 7)
import tensorflow as tf
from packaging import version
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import shutil
import keras_tuner as kt

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")
assert version.parse(tf.__version__) >= version.parse("2.8.0")


In [55]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [56]:
#norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1)
])

In [57]:
class MyClassificationHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp)

    def fit(self, hp, model, X, y, **kwargs):
        if hp.Boolean("normalize"):
            norm_layer = tf.keras.layers.Normalization()
            X = norm_layer(X)
        return model.fit(X, y, **kwargs)

In [58]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model

In [59]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)

In [60]:
bayesian_opt_tuner = kt.BayesianOptimization(
    MyClassificationHyperModel(), objective="val_accuracy", seed=14,
    max_trials=20, alpha=1e-4, beta=2.6,
    overwrite=True, directory="my_fashion_mnist", project_name="bayesian_opt")

bayesian_opt_tuner.search(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), callbacks=[early_stopping_cb])


Trial 20 Complete [00h 00m 40s]
val_accuracy: 0.9778000116348267

Best val_accuracy So Far: 0.9814000129699707
Total elapsed time: 00h 10m 24s


In [43]:
top3_models = bayesian_opt_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

In [44]:

top3_params = bayesian_opt_tuner.get_best_hyperparameters(num_trials=3)

print(top3_params[0].values)

{'n_hidden': 5, 'n_neurons': 236, 'learning_rate': 0.00015159319577885927, 'optimizer': 'adam', 'normalize': False}


In [45]:
best_model.fit(X_train_full, y_train_full, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 2.0002 - accuracy: 0.9795
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1841 - accuracy: 0.9883
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1339 - accuracy: 0.9879
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1021 - accuracy: 0.9874
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0738 - accuracy: 0.9876


In [46]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(test_accuracy)

313/313 [==============================] - 0s 982us/step - loss: 1.9449 - accuracy: 0.9273
0.927299976348877


In [47]:
best_model.save("my_best_keras_model92_7299976348877", save_format="tf")


INFO:tensorflow:Assets written to: my_best_keras_model92_7299976348877/assets


INFO:tensorflow:Assets written to: my_best_keras_model92_7299976348877/assets


In [48]:
model.save_weights("my_weights")

In [52]:
#model = tf.keras.models.load_model("my_best_keras_model92_7299976348877")
model.load_weights("my_weights")

ValueError: Received incompatible tensor with shape (50,) when attempting to restore variable with shape (236,) and name dense_1/bias:0.

In [53]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(test_accuracy)

313/313 [==============================] - 0s 1ms/step - loss: 1.9449 - accuracy: 0.9273
0.927299976348877
